In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
USE ROLE sysadmin;
USE WAREHOUSE ss_de_wh;

-- store table build 
-- ADD SS_STORES.csv

-- products table build 
CREATE OR REPLACE TABLE ss_101.raw_pos.referentiels_produit as 
select * EXCLUDE product_url from "ORGDATACLOUD$INTERNAL$SUMMIT_SPORTS_-_FRENCH_RETAIL_DATA_SET".SPORTS_DATA.SPORTS_PRODUCT_CATALOGUE;

-- orders table build 
CREATE OR REPLACE TABLE ss_101.raw_pos.order_detail as 
select * EXCLUDE store_name from "ORGDATACLOUD$INTERNAL$SUMMIT_SPORTS_-_FRENCH_RETAIL_DATA_SET".SPORTS_DATA.INSTORE_SALES_DATA_CRM3
where year(sale_date) < '2025';

-- customer loyalty table build 
CREATE OR REPLACE TABLE ss_101.raw_customer.customer_loyalty as 
select * from "ORGDATACLOUD$INTERNAL$SUMMIT_SPORTS_-_FRENCH_RETAIL_DATA_SET".SPORTS_DATA.CUSTOMERS;

In [ ]:
ALTER WAREHOUSE SS_DE_WH SET WAREHOUSE_SIZE = 'X-SMALL';

## View Aggregate Information and Inspect Query Profile

In [ ]:
SELECT 
    c.CUSTOMER_ID, c.FIRST_NAME, c.LAST_NAME, c.EMAIL, c.PHONE, c.REGISTRATION_DATE, c.PREFERRED_STORE, c.MARKETING_OPT_IN,
    round(COALESCE(SUM(isd.sales_price_euro - isd.discount_amount_euro) / COUNT(DISTINCT isd.order_id), 0),2) AS avg_basket_size,
    COALESCE(SUM(isd.sales_price_euro - isd.discount_amount_euro), 0) AS total_spend,
    COALESCE(COUNT(DISTINCT isd.order_id), 0) AS total_purchases,
    MAX(isd.sale_date) AS last_purchase_date,
    ARRAY_AGG(DISTINCT rp.product_name) WITHIN GROUP (ORDER BY rp.product_name) AS purchased_products
FROM ss_101.raw_customer.customer_loyalty c
LEFT JOIN ss_101.raw_pos.order_detail isd
    ON c.customer_id = isd.customer_id
LEFT JOIN SS_101.RAW_POS.REFERENTIELS_PRODUIT rp
    ON isd.product_id = rp.productid
where year(registration_date) < 2024 
GROUP BY c.CUSTOMER_ID, c.FIRST_NAME, c.LAST_NAME, c.EMAIL, c.PHONE, c.REGISTRATION_DATE, c.PREFERRED_STORE, c.MARKETING_OPT_IN;

## Create Dynamic Table for Orders

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE ss_101.harmonized.orders_v
    TARGET_LAG = '1 day'
    WAREHOUSE = ss_de_wh
    REFRESH_MODE = INCREMENTAL
    AS
    SELECT 
        od.ORDER_ID, od.STOREID, od.SALE_DATE, od.PRODUCT_ID, od.QUANTITY, od.SALES_PRICE_EURO, od.DISCOUNT_AMOUNT_EURO, od.PAYMENT_METHOD, od.SALES_ASSISTANT_ID, od.CUSTOMER_ID, od.CARD_ID,
        cl.FIRST_NAME, cl.LAST_NAME, 
        m.STORE_NAME, m.STORE_TYPE, m.POSTCODE,
        rp.PRODUCT_NAME, rp.BRAND, rp.MRP, rp.SALE_PRICE, rp.COLOUR, rp.PRODUCT_CATEGORY
    FROM ss_101.raw_pos.order_detail od
    LEFT JOIN ss_101.raw_customer.customer_loyalty cl ON od.customer_id = cl.customer_id
    LEFT JOIN ss_101.raw_pos.magasins m ON od.storeid = m.storeid
    LEFT JOIN ss_101.raw_pos.referentiels_produit rp ON od.product_id = rp.productid;

select * from ss_101.harmonized.orders_v
order by sale_date desc
limit 10 ;


In [ ]:
## graph streamlit

### Insert new data to inspect Dynamic Table Refresh

In [ ]:
INSERT INTO ss_101.raw_pos.order_detail(ORDER_ID, STOREID, SALE_DATE, PRODUCT_ID, QUANTITY, SALES_PRICE_EURO, DISCOUNT_AMOUNT_EURO, PAYMENT_METHOD, SALES_ASSISTANT_ID, CUSTOMER_ID, CARD_ID)
  SELECT ORDER_ID, STOREID, SALE_DATE, PRODUCT_ID, QUANTITY, SALES_PRICE_EURO, DISCOUNT_AMOUNT_EURO, PAYMENT_METHOD, SALES_ASSISTANT_ID, CUSTOMER_ID, CARD_ID
    FROM "ORGDATACLOUD$INTERNAL$SUMMIT_SPORTS_-_FRENCH_RETAIL_DATA_SET".SPORTS_DATA.INSTORE_SALES_DATA_CRM3
    where year(sale_date) = '2025';


ALTER DYNAMIC TABLE ss_101.harmonized.orders_v REFRESH COPY SESSION;

SELECT * FROM ss_101.harmonized.orders_v
order by sale_date desc
limit 5;

In [ ]:
## graph streamlit

## Create Analytics Views for Aggregations

In [ ]:
CREATE OR REPLACE VIEW SS_101.HARMONIZED.customer_loyalty_metrics_v AS
SELECT 
        c.CUSTOMER_ID, c.FIRST_NAME, c.LAST_NAME, c.EMAIL, c.PHONE, c.REGISTRATION_DATE, c.PREFERRED_STORE, c.MARKETING_OPT_IN,
        round(COALESCE(SUM(isd.sales_price_euro - isd.discount_amount_euro) / COUNT(DISTINCT isd.order_id), 0),2) AS avg_basket_size,
        COALESCE(SUM(isd.sales_price_euro - isd.discount_amount_euro), 0) AS total_spend,
        COALESCE(COUNT(DISTINCT isd.order_id), 0) AS total_purchases,
        MAX(isd.sale_date) AS last_purchase_date,
        ARRAY_AGG(DISTINCT rp.product_name) WITHIN GROUP (ORDER BY rp.product_name) AS purchased_products
    FROM ss_101.raw_customer.customer_loyalty c
    LEFT JOIN ss_101.raw_pos.order_detail isd
        ON c.customer_id = isd.customer_id
    LEFT JOIN SS_101.RAW_POS.REFERENTIELS_PRODUIT rp
        ON isd.product_id = rp.productid
    GROUP BY c.CUSTOMER_ID, c.FIRST_NAME, c.LAST_NAME, c.EMAIL, c.PHONE, c.REGISTRATION_DATE, c.PREFERRED_STORE, c.MARKETING_OPT_IN; 

/*--
 • analytics view creation
--*/

-- orders_v view
CREATE OR REPLACE VIEW ss_101.analytics.orders_v
COMMENT = 'summit sports Order Detail View'
    AS
SELECT DATE(o.sale_date) AS date, * FROM ss_101.harmonized.orders_v o;

-- customer_loyalty_metrics_v view
CREATE OR REPLACE VIEW ss_101.analytics.customer_loyalty_metrics_v
COMMENT = 'summit sports Customer Loyalty Member Metrics View'
    AS
SELECT * FROM ss_101.harmonized.customer_loyalty_metrics_v;

## Use Cortex AI to Enhance Analytics

In [ ]:
CREATE WAREHOUSE SS_DATASCIENCE WITH
  WAREHOUSE_TYPE = 'SNOWPARK-OPTIMIZED'
  WAREHOUSE_SIZE = MEDIUM
  RESOURCE_CONSTRAINT = 'MEMORY_16X_x86';

use warehouse SS_DATASCIENCE;

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")

In [ ]:
SELECT * FROM ss_101.harmonized.customer_loyalty_metrics_v
WHERE AI_FILTER(PROMPT('Basé sur le liste de produit acheté, cette personne a des enfants: {0}', ARRAY_TO_STRING(purchased_products, ', ')));

In [ ]:
use warehouse SS_DATASCIENCE;
CREATE OR REPLACE table SS_101.HARMONIZED.customer_loyalty_metrics_t AS
    select *,
    AI_COMPLETE('mistral-large',
        CONCAT('Vous etes un assistant marketing qui travaille pour un magasin de sport. En français, écrivez un email personalisé au client <clientname>', first_name, last_name,  '</clientname> qui donne une promo personalisé de max 10% sur une categorie qui leur interesse basé sur le liste de produits achetés :  <producthistory>', ARRAY_TO_STRING(purchased_products, ', '), '</producthistory>')
        ) as product_profile
    from SS_101.HARMONIZED.CUSTOMER_LOYALTY_WITH_METRICS_DT
    where avg_basket_size > 1000
    limit 3;

select * from SS_101.HARMONIZED.customer_loyalty_metrics_t;